# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [4]:
# If you're using colab, run this
# os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [5]:
llm = OpenAI(temperature=0)

In [6]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [7]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [8]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [9]:
from langchain_community.document_loaders import WebBaseLoader

In [10]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [11]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [12]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [13]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [14]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

<ipython-input-14-5458be099bf6>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [15]:
agent.invoke(
    "Does Ruff have a tool that could help automate code formatting to save time?"
)



> Entering new AgentExecutor chain...
 Yes, Ruff is a python linter, so it likely has some tools for code formatting.
Action: Ruff QA System
Action Input: "Does Ruff have a tool for code formatting?"
Observation:  Yes, Ruff has a formatter that is designed to be a drop-in replacement for Black.
Thought: This is helpful, but I wonder if there are any other options.
Action: Ruff QA System
Action Input: "What other options does Ruff have for code formatting?"
Observation:  Ruff can also replace Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade.
Thought: That's interesting, but I'm not sure if Ruff is the best option for code formatting.
Action: State of Union QA System
Action Input: "What is the current state of code formatting tools?"
Observation:  I don't know.
Thought: I should try asking a more specific question.
Action: State of Union QA System
Action Input: "What are the most popular code formatting tools currently in use?"
Observation:  I don't know.


{'input': 'Does Ruff have a tool that could help automate code formatting to save time?',
 'output': 'Ruff has several advantages for code formatting, including compatibility with Black, being a drop-in replacement for Flake8, implementing more rules than Pylint, automatically fixing lint violations, complementing type checkers, and replacing multiple tools. However, it also has limitations such as not supporting custom lint rules and potentially yielding different results than other tools. Ultimately, the best option for code formatting will depend on the specific needs and preferences of the user.'}

In [16]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter, so it may have different features and capabilities compared to flake8.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.

5. Can be used as a formatter: Ruff can be used as a fo

{'input': 'Why use ruff over flake8?',
 'output': 'The final answer is that Ruff may offer more features and compatibility compared to Flake8, making it a more versatile and efficient tool for linting and formatting Python code.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [17]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [18]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [19]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " I don't know."}

In [20]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.

5.

{'input': 'Why use ruff over flake8?',
 'output': '\nThere are a few reasons why someone might choose to use Ruff over Flake8:\n\n1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.\n\n2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.\n\n3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.\n\n4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.\n\n5. Can be used as a formatter: Ruff can be used as a formatter, making it a more versatile tool than Flake8.\n\nUltimately, the choice between Ruff and Flake8 will depend on the specific needs and preferences

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [21]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [22]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [23]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 We should check both tools to see if they have any information on Jupyter Notebooks.
Action: State of Union QA System
Action Input: "What did the president mention about Jupyter Notebooks in the state of the union?"
Observation:  I don't know.
Thought: The State of Union QA System doesn't seem to have any information on Jupyter Notebooks.
Action: Ruff QA System
Action Input: "What tool does ruff use to run over Jupyter Notebooks?"
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: It seems like Ruff does not directly use Jupyter Notebooks, but it does have some limitations when it comes to working with them.
Action: State of Union QA System
Action Input: "Did the president mention Jupyter Notebooks in the state of the union?"
Observation:  I don't know.
Thought: It doesn't seem like the president mentioned Jupyter Notebooks in the state of the union.
Thought: I now know the fi

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The president did not mention Jupyter Notebooks in the state of the union and Ruff does not directly use Jupyter Notebooks, but it does have some limitations when working with them.'}

In [24]:
agent.invoke(
    "Does Ruff have a tool that can help automate code format to save time?"
)



> Entering new AgentExecutor chain...
 Yes, Ruff is a python linter that can help with code formatting.
Action: Ruff QA System
Action Input: "Does Ruff have a tool that can help automate code format to save time?"
Observation:  Yes, Ruff has a formatter that can be used to automatically format code. It is designed to be a drop-in replacement for Black and aims to emit near-identical output when run over Black-formatted code. It can also be used alongside a linter, like Ruff's own linter or Flake8.
Thought: This is a useful tool for developers.
Action: State of Union QA System
Action Input: "What is the purpose of Ruff's code formatter?"
Observation:  The purpose of Ruff's code formatter is to automatically format code in a consistent and organized manner, making it easier to read and understand.
Thought: I now know the final answer.
Final Answer: The purpose of Ruff's code formatter is to automatically format code in a consistent and organized manner, making it easier to read and und

{'input': 'Does Ruff have a tool that can help automate code format to save time?',
 'output': "The purpose of Ruff's code formatter is to automatically format code in a consistent and organized manner, making it easier to read and understand."}